<a href="https://colab.research.google.com/github/habib-gm/Deep-Learning-Lab-AAU/blob/master/DL_Lab_4_class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch

## Creating Layers

In [2]:
class DenseLayer:
  # Layer initialization
  def __init__(self, n_inputs, n_neurons):
    # Initialize weights and biases
    self.weights = 0.01 * torch.rand(n_inputs, n_neurons)
    self.biases = torch.zeros((1, n_neurons))

  # Forward pass
  def forward(self, inputs):
    # Calculate output values from inputs, weights and biases
    self.output = torch.matmul(inputs, self.weights) + self.biases

## Activation Functions

### ReLU

In [3]:
class Activation_ReLU:
  # Forward pass
  def forward(self, inputs):
    self.output = torch.max(torch.tensor(0),inputs)

In [4]:
x = Activation_ReLU()
x.forward(torch.rand(2,3)-0.5)
x.output

tensor([[0.0000, 0.0000, 0.2900],
        [0.2115, 0.4666, 0.0000]])

### Sigmoid

In [5]:
class Activation_Sigmoid:
  # Forward pass
  def forward(self, inputs):
    self.output = 1 / (1 + torch.exp(inputs*-1))

In [6]:
x = Activation_Sigmoid()
x.forward(torch.rand(2,3))
x.output

tensor([[0.5854, 0.5256, 0.7084],
        [0.6750, 0.7252, 0.5251]])

### Softmax

In [7]:
class Activation_Softmax:
  # Forward pass
  def forward(self, inputs):
    # Get unnormalized probabilities
    exp_values = torch.exp(inputs - torch.max(inputs, axis=1, keepdim=True).values)
    # Normalize them for each sample
    probabilities = exp_values / torch.sum(exp_values, axis=1, keepdim=True)
    self.output = probabilities

In [8]:
x = Activation_Softmax()
x.forward(torch.rand(2,3))
print(x.output)
print(torch.sum(x.output,axis=1,keepdim=True))

tensor([[0.3992, 0.2752, 0.3256],
        [0.3197, 0.3660, 0.3143]])
tensor([[1.],
        [1.]])


## Loss

In [9]:
torch.clip(torch.tensor(0.015),torch.tensor(0.01),torch.tensor(0.05))

tensor(0.0150)

In [10]:
class Loss_CategoricalCrossentropy() :
  # Forward pass
  def forward(self, y_pred, y_true):
    samples = len(y_pred)
    # Clip data to prevent division by 0
    # Clip both sides to not drag mean towards any value
    y_pred_clipped = torch.clip(y_pred, 1e-8, 1 - 1e-8)
    # only if categorical labels
    if len(y_true.shape) == 1:
      correct_confidences = y_pred_clipped[range(samples), y_true]
    # Mask values - only for one-hot encoded labels
    elif len(y_true.shape) == 2:
      correct_confidences = torch.sum(y_pred_clipped * y_true, axis=1)
    log_loss = -torch.log(correct_confidences)
    data_loss = torch.mean(log_loss)
    return data_loss

In [11]:
softmax_outputs = torch.tensor([[0.7, 0.1, 0.2], [0.1, 0.5, 0.4],[0.02, 0.9, 0.08]])
class_targets = torch.tensor([[1, 0, 0], [0, 1, 0], [1, 0, 0]])

In [12]:
x = Loss_CategoricalCrossentropy()
x.forward(softmax_outputs, class_targets)

tensor(1.6539)

## Accuracy

In [13]:
class Accuracy():
  def calculate(self, y_pred, y_true):
    predictions = torch.argmax(y_pred, axis=1)
    if len(y_true.shape) == 2:
      y_true = torch.argmax(y_true, axis=1)
    accuracy = torch.mean((predictions == y_true).float())
    return accuracy

In [14]:
x = Accuracy()
x.calculate(softmax_outputs, class_targets)

tensor(0.6667)

## Example 1
### Preparing Dataset

In [15]:
# Create dataset
X = torch.rand((4,2))
y = torch.tensor([0,1,1,0])

## Creating model

In [16]:
dense1 = DenseLayer(2, 4) # 2 input features and 3 output values
activation1 = Activation_ReLU()
dense2 = DenseLayer(4, 4)
activation2 = Activation_ReLU()
dense3 = DenseLayer(4, 2)
activation3 = Activation_Softmax()

## Loss and accuracy

In [17]:
loss_function = Loss_CategoricalCrossentropy()
acc = Accuracy()

## Forward pass

In [18]:
dense1.forward(X)
activation1.forward(dense1.output)
dense2.forward(activation1.output)
activation2.forward(dense2.output)
dense3.forward(activation2.output)
activation3.forward(dense3.output)

## Displaying results

In [19]:
print(activation3.output)
loss = loss_function.forward(activation3.output, y)
accuracy = acc.calculate(activation3.output,y)
print('loss:', loss)
print("Accuracy:", accuracy)

tensor([[0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000]])
loss: tensor(0.6931)
Accuracy: tensor(0.5000)


## Example 2
### testing our model on real dataset

In [20]:
import numpy as np
from sklearn.datasets import load_iris

In [21]:
# Load the Iris dataset from scikit-learn
iris = load_iris()
X = iris.data
y = iris.target

In [22]:
# Convert the NumPy arrays to PyTorch tensors
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.int64)

In [23]:
print("X shape:", X.shape)
print("y shape:", y.shape)
print("Feature names:", iris.feature_names)
print("Class names:", iris.target_names)
print(X[:5])
print(y[:5])

X shape: torch.Size([150, 4])
y shape: torch.Size([150])
Feature names: ['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']
Class names: ['setosa' 'versicolor' 'virginica']
tensor([[5.1000, 3.5000, 1.4000, 0.2000],
        [4.9000, 3.0000, 1.4000, 0.2000],
        [4.7000, 3.2000, 1.3000, 0.2000],
        [4.6000, 3.1000, 1.5000, 0.2000],
        [5.0000, 3.6000, 1.4000, 0.2000]])
tensor([0, 0, 0, 0, 0])


In [24]:
class ClassificationModel():

  def __init__(self, num_of_features, num_of_class):
    # creating the model
    self.dense1 = DenseLayer(num_of_features,16)
    self.activation1 = Activation_ReLU()
    self.dense2 = DenseLayer(16, 16)
    self.activation2 = Activation_ReLU()
    # self.dense3 = DenseLayer(16, num_of_class)
    self.output_layer = DenseLayer(16, num_of_class)
    self.activation3 = Activation_Softmax()
    # loss and accuracy
    self.loss_function = Loss_CategoricalCrossentropy()
    self.acc = Accuracy()

  def model(self, X, y):
    self.y = y
    # forward pass
    self.dense1.forward(X)
    self.activation1.forward(self.dense1.output)
    self.dense2.forward(self.activation1.output)
    self.activation2.forward(self.dense2.output)
    # self.dense3.forward(self.activation2.output)
    self.output_layer.forward(self.activation2.output)
    self.activation3.forward(self.output_layer.output)

  def loss_and_accuracy(self):
    self.loss = self.loss_function.forward(self.activation3.output, self.y)
    self.accuracy = self.acc.calculate(self.activation3.output,self.y)

In [25]:
test = ClassificationModel(4, 3)

In [26]:
test.model(X, y)
test.loss_and_accuracy()

In [27]:
print('loss:', test.loss)
print("Accuracy:", test.accuracy)

loss: tensor(1.0986)
Accuracy: tensor(0.3333)


###  How can we adjust the weights and biases to decrease the loss?


#### Option 1: randomly changing the weights, checking the loss, and repeating this until the lowest loss found.



In [28]:
test_iris = ClassificationModel(4, 3)

In [29]:
torch.set_printoptions(precision=10)

In [30]:
lowest_loss = torch.tensor(99999999)

In [31]:
for iteration in range(10000):

  # Perform a forward pass
  test_iris.model(X,y)
  test_iris.loss_and_accuracy()
  loss = test_iris.loss
  accuracy = test_iris.accuracy

  # If loss is smaller - print and save weights and biases aside
  if loss < lowest_loss:
    print('New set of weights found, iteration:', iteration, 'loss:', loss, 'acc:', accuracy)
    best_dense1_weights = test_iris.dense1.weights
    best_dense1_biases = test_iris.dense1.biases
    best_dense2_weights = test_iris.dense2.weights
    best_dense2_biases = test_iris.dense2.biases
    # best_dense3_weights = test_iris.dense3.weights
    # best_dense3_biases = test_iris.dense3.biases
    best_output_layer_weights = test_iris.output_layer.weights
    best_output_layer_biases = test_iris.output_layer.biases
    lowest_loss = loss

  # Generate a new set of weights for iteration
  test_iris.dense1.weights = 0.01 * torch.rand(4, 16)
  test_iris.dense1.biases = 0.01 * torch.rand(1, 16)
  test_iris.dense2.weights = 0.01 * torch.rand(16, 16)
  test_iris.dense2.biases = 0.01 * torch.rand(1, 16)
  # test_iris.dense3.weights = 0.01 * torch.rand(16, 3)
  # test_iris.dense3.biases = 0.01 * torch.rand(1, 3)
  test_iris.output_layer.weights = 0.01 * torch.rand(16, 3)
  test_iris.output_layer.biases = 0.01 * torch.rand(1, 3)

New set of weights found, iteration: 0 loss: tensor(1.0986144543) acc: tensor(0.3333333433)
New set of weights found, iteration: 1 loss: tensor(1.0986094475) acc: tensor(0.3333333433)
New set of weights found, iteration: 3 loss: tensor(1.0986047983) acc: tensor(0.3333333433)
New set of weights found, iteration: 6 loss: tensor(1.0986040831) acc: tensor(0.3333333433)
New set of weights found, iteration: 24 loss: tensor(1.0985995531) acc: tensor(0.3333333433)
New set of weights found, iteration: 152 loss: tensor(1.0985987186) acc: tensor(0.3333333433)
New set of weights found, iteration: 213 loss: tensor(1.0985937119) acc: tensor(0.3333333433)
New set of weights found, iteration: 511 loss: tensor(1.0985903740) acc: tensor(0.3333333433)
New set of weights found, iteration: 825 loss: tensor(1.0985870361) acc: tensor(0.3333333433)
New set of weights found, iteration: 4729 loss: tensor(1.0985864401) acc: tensor(0.3333333433)


In [32]:
print('loss:', test_iris.loss)
print("Accuracy:", test_iris.accuracy)

loss: tensor(1.0986151695)
Accuracy: tensor(0.3333333433)


#### Option 2:  instead of setting parameters with randomly-chosen values each iteration, apply a fraction of these values to parameters.

In [33]:
test_iris = ClassificationModel(4, 3)

In [34]:
lowest_loss = torch.tensor(99999999)

In [40]:
for iteration in range(10000):

  # Perform a forward pass
  test_iris.model(X,y)
  test_iris.loss_and_accuracy()
  loss = test_iris.loss
  accuracy = test_iris.accuracy

  # If loss is smaller - print and save weights and biases aside
  if loss < lowest_loss:
    print('New set of weights found, iteration:', iteration, 'loss:', loss, 'acc:', accuracy)
    best_dense1_weights = test_iris.dense1.weights
    best_dense1_biases = test_iris.dense1.biases
    best_dense2_weights = test_iris.dense2.weights
    best_dense2_biases = test_iris.dense2.biases
    # best_dense3_weights = test_iris.dense3.weights
    # best_dense3_biases = test_iris.dense3.biases
    best_output_layer_weights = test_iris.output_layer.weights
    best_output_layer_biases = test_iris.output_layer.biases
    lowest_loss = loss
  else:
    test_iris.dense1.weights = best_dense1_weights
    test_iris.dense1.biases = best_dense1_biases
    test_iris.dense2.weights = best_dense2_weights
    test_iris.dense2.biases = best_dense2_biases
    # test_iris.dense3.weights = best_dense3_weights
    # test_iris.dense3.biases = best_dense3_biases
    test_iris.output_layer.weights = best_output_layer_weights
    test_iris.output_layer.biases = best_output_layer_biases

  # Generate a new set of weights for iteration
  test_iris.dense1.weights += 0.05 * torch.rand(4, 16)
  test_iris.dense1.biases += 0.05 * torch.rand(1, 16)
  test_iris.dense2.weights += 0.05 * torch.rand(16, 16)
  test_iris.dense2.biases += 0.05 * torch.rand(1, 16)
  # test_iris.dense3.weights += 0.05 * torch.rand(16, 3)
  # test_iris.dense3.biases += 0.05 * torch.rand(1, 3)
  test_iris.output_layer.weights += 0.05 * torch.rand(16, 3)
  test_iris.output_layer.biases += 0.05 * torch.rand(1, 3)

New set of weights found, iteration: 1 loss: tensor(1.0976145267) acc: tensor(0.3333333433)


#### Option 3:  using optimization

In [41]:
X = torch.tensor([0.1, 0.5])
y = torch.tensor([0.05, 0.95])

In [42]:
hidden_layer_1 = DenseLayer(2, 2)
activation_1 = Activation_ReLU()
output_layer = DenseLayer(2, 2)
activation2 = Activation_Sigmoid()

In [44]:
def forward_pass(X):
  hidden_layer_1.forward(X)
  activation1.forward(hidden_layer_1.output)
  output_layer.forward(activation1.output)
  activation2.forward(output_layer.output)
  return activation2.output

In [45]:
def back_prop(fp):
  lr = torch.tensor(0.01)
  back1 = (fp[0][0]-y[0])*(1-fp[0][0])*fp[0][0]
  back2 = (fp[0][1]-y[1])*(1-fp[0][1])*fp[0][1]
  output_layer.weights[0][0] -= lr * back1*activation1.output[0][0]
  output_layer.weights[0][1] -= lr * back1*activation1.output[0][1]
  output_layer.weights[1][0] -= lr * back2*activation1.output[0][0]
  output_layer.weights[1][1] -= lr * back2*activation1.output[0][1]
  output_layer.biases[0][0] -= lr * back1
  output_layer.biases[0][1] -= lr * back2

  hidden_layer_1.weights[0][0] -= lr * (back1 * output_layer.weights[0][0] * X[0] + back2 * output_layer.weights[0][1] * X[0] ) if hidden_layer_1.output[0][0] > 0 else 0
  hidden_layer_1.weights[0][1] -= lr * (back1 * output_layer.weights[0][0] * X[1] + back2 * output_layer.weights[0][1] * X[1] ) if hidden_layer_1.output[0][0] > 0 else 0
  hidden_layer_1.weights[1][0] -= lr * (back1 * output_layer.weights[1][0] * X[0] + back2 * output_layer.weights[1][1] * X[0] ) if hidden_layer_1.output[0][1] > 0 else 0
  hidden_layer_1.weights[1][1] -= lr * (back1 * output_layer.weights[1][0] * X[1] + back2 * output_layer.weights[1][1] * X[1] ) if hidden_layer_1.output[0][1] > 0 else 0
  hidden_layer_1.biases[0][0] -= lr * (back1 * output_layer.weights[0][0] + back2 * output_layer.weights[0][1] ) if hidden_layer_1.output[0][0] > 0 else 0
  hidden_layer_1.biases[0][1] -= lr * (back1 * output_layer.weights[1][0] + back2 * output_layer.weights[1][1] ) if hidden_layer_1.output[0][1] > 0 else 0


In [46]:
def error_calculation(y_true, y_pred):
  return torch.mean(0.5*(y_true - y_pred)**2)

In [47]:
loss = 0.0001

In [49]:
y_pred = forward_pass(X)
err = error_calculation(y, y_pred)
print("Initial loss:", err)
print("Initial prediction:",y_pred)

while err > loss:
  back_prop(y_pred)
  y_pred = forward_pass(X)
  err = error_calculation(y, y_pred)
print("Final loss:", err)
print("Final prediction:",y_pred)
print("Target value:",y)

Initial loss: tensor(9.9997625512e-05)
Initial prediction: tensor([[0.0641419590, 0.9358580112]])
Final loss: tensor(9.9997625512e-05)
Final prediction: tensor([[0.0641419590, 0.9358580112]])
Target value: tensor([0.0500000007, 0.9499999881])
